#Using deep features to build an image classifier

#Fire up GraphLab Create

In [1]:
import graphlab

#Load a common image analysis dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set.  

In [2]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

This non-commercial license of GraphLab Create for academic use is assigned to niklas.forck@zalando.de and will expire on January 20, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1487936581.log
INFO:graphlab.cython.cy_server:GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1487936581.log


#Exploring the image data

In [3]:
graphlab.canvas.set_target('ipynb')

In [4]:
image_train['image'].show()

#Train a classifier on the raw image pixels

We first start by training a classifier on just the raw pixels of the image.

In [5]:
raw_pixel_model = graphlab.logistic_classifier.create(image_train,target='label',
                                              features=['image_array'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 1898

...

| 6         | 12       | 1.000000  | 7.174147     | 0.455743          | 0.401869            |

| 10        | 16       | 1.000000  | 9.546457     | 0.506849          | 0.514019            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

#Make a prediction with the simple model based on raw pixels

In [6]:
image_test[0:3]['image'].show()

In [7]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [8]:
raw_pixel_model.predict(image_test[0:3])

dtype: str
Rows: 3
['dog', 'cat', 'bird']

The model makes wrong predictions for all three images.

#Evaluating raw pixel model on test data

In [9]:
raw_pixel_model.evaluate(image_test)

{'accuracy': 0.47075, 'auc': 0.7131040833333333, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     dog      |       cat       |  188  |
 |     cat      |       cat       |  266  |
 |     bird     |       dog       |  264  |
 |     bird     |       cat       |  123  |
 |     cat      |       dog       |  369  |
 |     dog      |       dog       |  497  |
 |     dog      |    automobile   |   85  |
 |     cat      |    automobile   |  140  |
 |     dog      |       bird      |  230  |
 |  automobile  |    automobile   |  608  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.46727366864016373, 'log_loss': 1.2361583926917856, 'precision': 

The accuracy of this model is poor, getting only about 46% accuracy.

#Can we improve the model using deep features

We only have 2005 data points, so it is not possible to train a deep neural network effectively with so little data.  Instead, we will use transfer learning: using deep features trained on the full ImageNet dataset, we will train a simple model on this small dataset.

In [10]:
len(image_train)

2005

##Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [11]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

As we can see, the column deep_features already contains the pre-computed deep features for this data. 

In [12]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


#Given the deep features, let's train a classifier

In [13]:
deep_features_model = graphlab.logistic_classifier.create(image_train,
                                                         features=['deep_features'],
                                                         target='label')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'deep_features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

...

| 9         | 17       | 0.144936  | 12.801455    | 0.835718          | 0.779661            |

| 10        | 18       | 0.144936  | 13.655375    | 0.850556          | 0.779661            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

#Apply the deep features model to first few images of test set

In [14]:
image_test[0:3]['image'].show()

In [15]:
deep_features_model.predict(image_test[0:3])

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

The classifier with deep features gets all of these images right!

#Compute test_data accuracy of deep_features_model

As we can see, deep features provide us with significantly better accuracy (about 78%)

In [16]:
deep_features_model.evaluate(image_test)

{'accuracy': 0.776, 'auc': 0.9384929166666701, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  automobile  |       cat       |   12  |
 |  automobile  |    automobile   |  953  |
 |     bird     |    automobile   |   30  |
 |     dog      |    automobile   |   20  |
 |     dog      |       dog       |  797  |
 |     cat      |       dog       |  308  |
 |     cat      |       bird      |   64  |
 |     bird     |       dog       |   78  |
 |  automobile  |       bird      |   26  |
 |     bird     |       bird      |  761  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.7746862814863091, 'log_loss': 0.6096394615607934, 'precision': 0.7

## Homework

In [53]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [17]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [18]:
image_dog = image_train[image_train['label'] == 'dog']

In [21]:
image_cat = image_train[image_train['label'] == 'cat']

In [22]:
image_bird = image_train[image_train['label'] == 'bird']

In [23]:
image_automobile = image_train[image_train['label'] == 'automobile']

In [24]:
dog_model = graphlab.nearest_neighbors.create(image_dog,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [25]:
cat_model = graphlab.nearest_neighbors.create(image_cat,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [26]:
bird_model = graphlab.nearest_neighbors.create(image_bird,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [27]:
automobile_model = graphlab.nearest_neighbors.create(image_automobile,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [55]:
cats = cat_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 40.221ms     |

| Done         |         | 100         | 137.146ms    |

+--------------+---------+-------------+--------------+

In [59]:
get_images_from_ids(cats[0])['image'].show()

In [60]:
dogs = dog_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 29.228ms     |

| Done         |         | 100         | 153.797ms    |

+--------------+---------+-------------+--------------+

In [61]:
get_images_from_ids(dogs[0])['image'].show()

In [31]:
cat_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 25.789ms     |

| Done         |         | 100         | 103.416ms    |

+--------------+---------+-------------+--------------+

36.15573070978294

In [32]:
dog_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 23.671ms     |

| Done         |         | 100         | 103.613ms    |

+--------------+---------+-------------+--------------+

37.77071136184156

In [33]:
image_test_dog = image_test[image_test['label'] == 'dog']

In [34]:
image_test_cat = image_test[image_test['label'] == 'cat']

In [35]:
image_test_bird = image_test[image_test['label'] == 'bird']

In [36]:
image_test_automobile = image_test[image_test['label'] == 'automobile']

In [37]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 424.066ms    |

| Done         | 509000  | 100         | 472.073ms    |

+--------------+---------+-------------+--------------+

In [38]:
dog_dog_neighbors = dog_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 418.944ms    |

| Done         | 509000  | 100         | 433.518ms    |

+--------------+---------+-------------+--------------+

In [39]:
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 119000  | 24.8954     | 397.435ms    |

| Done         | 478000  | 100         | 432.484ms    |

+--------------+---------+-------------+--------------+

In [40]:
dog_automobile_neighbors = automobile_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 128000  | 25.1473     | 391.849ms    |

| Done         | 509000  | 100         | 438.944ms    |

+--------------+---------+-------------+--------------+

In [41]:
dog_automobile_neighbors.show()

In [42]:
dog_distances = graphlab.SFrame({'dog-automobile': dog_automobile_neighbors['distance'],'dog-bird': dog_bird_neighbors['distance'],'dog-cat': dog_cat_neighbors['distance'],'dog-dog': dog_dog_neighbors['distance'] })

In [43]:
dog_distances.show()

In [46]:
def is_dog_correct(row):
    dog_dis = row['dog-dog']
    if dog_dis > row['dog-bird']:
        return 0
    if dog_dis > row['dog-automobile']:
        return 0
    if dog_dis > row['dog-cat']:
        return 0
    return 1
    

In [50]:
dog_distances.apply(is_dog_correct).sum()

678

In [52]:
dog_distances.apply(is_dog_correct).sum() / float(len(dog_distances))

0.678